In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

**Задание 2**

а)

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.4)
X = vectorizer.fit_transform(data.comment)

In [4]:
cosine_similarity(X[3], X[12666])

array([[0.50771793]])

Ответ: 0.50771793

б)

In [5]:
for i in cosine_similarity(X[43], X).argsort()[0, -4:]:
    print(i, data.loc[i, 'comment'], cosine_similarity(X[43], X[i]))

1957 Че за бригада и че за махоун? Из полицейской академии? [[0.23279247]]
14325 Ну оль, ну вот че тебе надо? Иди в по сри, а этот достопочтенный тред оставь в покое
 [[0.25324249]]
2245 Че ты там мямлишь, утырок? Сопли подотри
 [[0.27934598]]
43 Люди зажрались и охуели если по мнению этих игроков андромеда лучше Антема. Хотя че там с багаутом76 сравнивают вон... Вот оно че оказывается, игроки просто охуели, ну ок. Вот долбанные пидерасы, не хотят покупать такую отличную игру, а еще смеют ругать такой божественный геймплей. Баги, хуевый геймплей поправят, а контент, ну его запилят, через год другой, причем бесплатно! Ага. Стоит лишь потерпеть!
 [[1.]]


На самом деле эти тексты не очень близки к тексту 43. Но у них и значения близости невысокое, так что, видимо, у 43 текста просто нет сильно похожих в датасете.

**Задание 3**

In [6]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

Обучим два классификатора: логистическую регрессию с tf-idf векторами и наивный байес с векторами абсолютных значений частотности.

In [7]:
vectorizer1 = TfidfVectorizer(strip_accents='unicode', ngram_range=(1,2), min_df=5, max_df=0.4, binary=True)
X = vectorizer1.fit_transform(train.comment)
X_test = vectorizer1.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

clf1 = LogisticRegression(C=1, class_weight='balanced')
clf1.fit(X, y)
preds = clf1.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.91      0.89      0.90       985
         1.0       0.77      0.81      0.79       457

    accuracy                           0.86      1442
   macro avg       0.84      0.85      0.85      1442
weighted avg       0.87      0.86      0.87      1442



In [8]:
vectorizer2 = CountVectorizer(strip_accents='unicode', max_df=0.5, min_df=0.0001, analyzer='word', ngram_range=(1,1))
X = vectorizer2.fit_transform(train.comment)
X_test = vectorizer2.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

clf2 = MultinomialNB(alpha=0.5, fit_prior=False)
clf2.fit(X, y)
preds = clf2.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.93      0.91      0.92       985
         1.0       0.81      0.85      0.83       457

    accuracy                           0.89      1442
   macro avg       0.87      0.88      0.87      1442
weighted avg       0.89      0.89      0.89      1442



Теперь предскажем токсичность для 3000 текстов из корпуса двача и выведем десять самых токсичных:

In [9]:
with open('2ch_corpus.txt', 'r', encoding='utf-8') as f:
    dvach = f.readlines()[:3000]

Первым логрег-классификатором:

In [10]:
X_test_dvach_1 = vectorizer1.transform(dvach)
probas1 = clf1.predict_proba(X_test_dvach_1)

In [11]:
tox_probs = list(zip([x for x in range(len(probas1))], [x[1] for x in probas1]))
toxic_ids = [x[0] for x in sorted(tox_probs, key=lambda x: x[1], reverse=True)[:10]]
texts = [dvach[i] for i in toxic_ids]
texts

[' Ты превращаешься в Суисейсеки, десу.\n',
 ' >ElixirНу ты и говноед.\n',
 ' Крысу ты разделовал?\n',
 ' Ебать ты юморист донный.\n',
 ' Кто подданный? Ты подданый, ёпт. Здравствуй.\n',
 ' Просто ты скучный и предсказуемый.\n',
 ' Ты автор - тебе видней.\n',
 ' И нахуй ты это ему написал?\n',
 ' Ты зацени какой подклад на пиджаке\n',
 '   остановись, ты пишешь хуйню\n']

И вторым байесовским классификатором:

In [12]:
X_test_dvach_2 = vectorizer2.transform(dvach)
probas2 = clf2.predict_proba(X_test_dvach_2)

In [13]:
tox_probs = list(zip([x for x in range(len(probas2))], [x[1] for x in probas2]))
toxic_ids = [x[0] for x in sorted(tox_probs, key=lambda x: x[1], reverse=True)[:10]]
texts = [dvach[i] for i in toxic_ids]
texts

[' Ёбаные советские названия блять. КРУЖКИ блять. Всегда ненавидел. Блять. Почему бы не сказать КЛУБ ПО ИНТЕРЕСАМ, как у япошек, нет блять, будем гуманитарную хуйню, сравнения блять, типа В КРУГ СОБРАЛИСЬ, ахахахаха кружок ахахахахаха))))) пиздец блять. Может при Сталине это звучало, но уже при Горбачёве это просто бесило, а сейчас это выглядит вообще как атавизм. Или ещё ебанутое слово СЛЁТ блять. Сука, СЛЁТ, почему слёт, а не съезд? Типа съезд - это для КПСС, не доросли ещё? Или типа в Совке так всё пиздато, что даже у школия есть своё самолёт? СЛЁТ сука, мы что, блять, стрижи? Или это какая-то аллегория на ёбаных ОРЛЯТ? Вот тоже заебучее сравнение, всегда бесило, ОРЯЛА УЧАТСЯ ЛЕТАТЬ, блять, да мне похуй на каких-то куриц, орлят, блять, голубей, петухов, учатся они летать блять, а страусы вон не учатся, мне-то что до ваших сраных ОРЛЯТ, пел всегда КОЛЗЛЯТ, да-да, школие, этот прикол существовал ещё до Задорнова и прочих клованов - в совковых песнях заменять "орёл" на "козёл". Ну до ч

Результаты получились совсем разные: во-первых, бросается в глаза, что логрег выбрал короткие тексты, а байес - больше длинных. Даже не знаю, как это можно объяснить. Во-вторых, тексты из второго набора гораздо больше подходят под описание токсичности, чем тексты из первого, хотя и в первом такие встречаются; возможно, это связано с тем, что второй классификатор показал лучшие результаты на тестовой выборке и в целом получился точнее.